In [1]:
# -------------------------------------------------
# Initial import.
# -------------------------------------------------

import numpy as np
import matplotlib,pylab as plt
import tensorflow as tf
import os
import keras
from scipy.io import loadmat 

from keras.layers import Input, Dense, Activation, Embedding, LSTM, Flatten

from keras.models import Model, Sequential
from keras.utils import np_utils


/home/mlvm2/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# --------------------------------------------------
# PATH of where data saved.
# Create Data list, wait for processing.
# --------------------------------------------------

PATH = "/home/mlvm2/dissertation/My_programme/dissertation_data"
Data_list = list(os.path.join(PATH, name) for name in os.listdir(PATH))


In [3]:
# --------------------------------------------------
# Average Abs Feature Extraction Function.
# --------------------------------------------------
def avg_abs_extract(data_path):
    
    # load data first.
    dataset = loadmat(data_path)
    
    signals = dataset['emg'][0:125]
    
    # first ensure how many row samples inthe data should be cut. 
    cut_num = len(signals[0])%40
    #print(cut_num)
    
    # cut signal.
    cut_signals = signals.copy()
    for i in range(0, cut_num):
        cut_signals = np.delete(cut_signals, len(cut_signals[0]) - 1, axis = 1)
    signal_process1 = np.zeros(shape = [cut_signals.shape[0], int(cut_signals.shape[1] / 10)])
    #print(cut_signals.shape)
    
    # extract abs features.
    for channels in range(0, cut_signals.shape[0]):
        process1 = np.zeros(int(cut_signals.shape[1] / 10))
        for i in range(0, int(cut_signals.shape[1])/ 10):
            for j in range(0, 10):
                process1[i] = process1[i] + abs(cut_signals[channels, 10 * i + j])
            process1[i] = process1[i] / 10
        for i in range(0, int(cut_signals.shape[1] / 10)):
            signal_process1[channels][i] = process1[i]
    
    signal_process2 = np.zeros(shape = [125, len(process1)/4, 4])
    for channel in range(0,125):
        for i in range(0, len(process1)/4):
            for j in range(0,4):
                signal_process2[channel, i, j] = signal_process1[channel, i*4+j]
    
    signal_input  = signal_process2.copy()
    signal_input_final = np.zeros(shape = [len(process1)/4, 125, 4])
    for i in range(0, len(process1)/4):
        for j in range(0, 125):
            for k in range(0, 4):
                signal_input_final[i,j,k] = signal_input[j,i,k]
    #print(signal_input_final.shape)
                
    cut_num2 = (len(process1)/4) % 10
    for i in range(0, cut_num2):
        signal_input_final = np.delete(signal_input_final, signal_input_final.shape[0] - 1, axis = 0)
    signal_input_final = signal_input_final.reshape(((len(process1)/4) - cut_num2)/10, 10, 125*4)
                    
    return signal_input_final


In [4]:
def label_extract(data_path):
    
    dataset = loadmat(data_path)
    
    labels = dataset['glove'][0]

    cut_num = len(labels)%40
    
    for i in range(0, cut_num):
        cut_labels = np.delete(labels, len(labels) - 1)

    labels_process1 = np.zeros(int(len(cut_labels)/40))

    for i in range(int(len(cut_labels)/40)):
        labels_process1[i] = labels[i*40]
    
    cut_num2 = len(labels_process1)%10
    for i in range(0, cut_num2):
        labels_process1 = np.delete(labels_process1, len(labels_process1) - 1)
    
    labels_process2 = np.zeros(int(len(labels_process1)/10))
    
    for i in range(int(len(labels_process1)/10)):
        labels_process2[i] = labels_process1[i*10]
    
    return labels_process2

In [16]:
i = 9
signal = avg_abs_extract(Data_list[i])
label = label_extract(Data_list[i])
np.save("t_s_signal_9_10input", signal)
np.save("t_s_label_9_10input",label)